**Tarea 7 - Integracion Taylor**

Estudiantes: Yuriko Yamamoto, Ignacio Vargas

**Ejercicio 1:** Usando su implementación de polinomios de Taylor, escriban un integrador para la ecuación diferencial que se desarrolló en este ejemplo, esto es, con la condición inicial x(0)=3.

El integrador debe hacer las operaciones necesarias para obtener automáticamente los coeficientes x[k], en cada paso de integración, a partir de la condición inicial local. Un requisito básico para esto es que tengan una implementación de la función Pα(x)=[g(x)]α con g(x) un polinomio, que obtuvieron en la "Tarea6", y que funcione bien en particular para alpha::Int.

La implementación debe consistir de varias funciones: 

- Una función donde se calculen los coeficientes $x_{[k]}(t_0)$ de la expansión. Esta función deberá llamar a otra donde se implementan las ecuaciones de movimiento.

- Una función donde se obtenga el paso de integración $h$.

- Otra función donde se haga la suma por el método de Horner.

- Finalmente, una función que use las funciones anteriores para hacer la integración desde un tiempo inicial a uno final. En este punto, *fingiremos ignorancia*, en el sentido que el tiempo inicial es cero, y el tiempo final será $0.5$ (que está más allá de donde hay problemas).


Dado que conocemos la solución analítica, den como función de $t$ el error relativo de su integrador (respecto al valor del resultado analítico).

In [ ]:
#include("TaylorInt.jl")
#import <modulo>

In [1]:
type Taylor{T<:Number}
    
    coeficientes :: Array{T,1}
    orden :: T
    
    #Constructor interno
    function Taylor(coeficientes::Array{T,1}, orden::T)
        
        longitud_coeficientes= length(coeficientes)
        orden = max(orden, longitud_coeficientes-1)
        
        vector = zeros(T, orden+1)
        
        vector[1:longitud_coeficientes] = coeficientes[1:end]
        new(vector, orden)
        
    end
end



In [2]:
#Constructor externo
Taylor{T<:Number}(x::Taylor{T}, orden::Integer) = Taylor{T}(x.coeficientes, orden)
#Taylor{T<:Number}(x::Taylor{T}) = Taylor{T}(x.coeficientes, x.orden)
Taylor{T<:Number}(x::Taylor{T}) = x
Taylor{T<:Number}(coeficientes::Array{T,1}, orden::Integer) = Taylor{T}(coeficientes, orden)
Taylor{T<:Number}(coeficientes::Array{T,1}) = Taylor{T}(coeficientes, length(coeficientes)-1)
Taylor{T<:Number}(x::T, orden::Integer) = Taylor{T}([x], orden)
Taylor{T<:Number}(x::T) = Taylor{T}([x], 0)

Taylor{T<:Number}

In [3]:
#Tipo, longitud
eltype{T<:Number}(::Taylor{T}) = T
length{T<:Number}(a::Taylor{T}) = a.orden

length (generic function with 1 method)

In [4]:
#Conversion y reglas promocion
convert{T<:Number}(::Type{Taylor{T}}, a::Taylor) = Taylor(convert(Array{T,1}, a.coeficientes), a.orden)
convert{T<:Number, S<:Number}(::Type{Taylor{T}}, b::Array{S,1}) = Taylor(convert(Array{T,1},b))
convert{T<:Number, S<:Number}(::Type{Taylor{T}}, b::S) = Taylor([convert(T,b)], 0)

promote_rule{T<:Number, S<:Number}(::Type{Taylor{T}}, ::Type{Taylor{S}}) = Taylor{promote_type(T, S)}
promote_rule{T<:Number, S<:Number}(::Type{Taylor{T}}, ::Type{Array{S,1}}) = Taylor{promote_type(T, S)}
promote_rule{T<:Number, S<:Number}(::Type{Array{S,1}}, ::Type{Taylor{T}}) = Taylor{promote_type(T, S)}
promote_rule{T<:Number, S<:Number}(::Type{Taylor{T}}, ::Type{S}) = Taylor{promote_type(T, S)}
promote_rule{T<:Number, S<:Number}(::Type{S}, ::Type{Taylor{T}}) = Taylor{promote_type(T, S)}

promote_rule (generic function with 5 methods)

In [5]:
# #Funcion auxiliar
function firstnonzero{T<:Number}(a::Taylor{T})
    orden = a.orden
    nonzero = orden+1
    z = zero(T)
    for i = 1:orden+1
        if a.coeficientes[i] != z
            nonzero = i-1
            break
        end
    end
    nonzero
end
function fixshape{T<:Number, S<:Number}(a::Taylor{T}, b::Taylor{S})
    orden = max(a.orden, b.orden)
    a1, b1 = promote(a, b)
    return Taylor(a1, orden), Taylor(b1, orden), orden
end

fixshape (generic function with 1 method)

In [6]:
#Zero y uno
zero{T<:Number}(a::Taylor{T}) = Taylor(zero(T), a.orden)
one{T<:Number}(a::Taylor{T}) = Taylor(one(T), a.orden)



one (generic function with 1 method)

In [7]:
import Base: +, -, *, /, ==, length

#Igualdad
function ==(a::Taylor, b::Taylor)
    a1, b1, orden = fixshape(a, b)
    return a1.coeficientes == b1.coeficientes
end
==(a::Taylor, b::Number) = ==(a, Taylor(b, a.orden))
==(a::Number, b::Taylor) = ==(b, Taylor(a, b.orden))

#Suma y resta
for f in (:+, :-)
    @eval begin
        function ($f)(a::Taylor, b::Taylor)
            a1, b1, orden = fixshape(a, b)
            v = ($f)(a1.coeficientes, b1.coeficientes)
            return Taylor(v, orden)
        end
       ($f)(a::Taylor, b::Number) = ($f)(a, Taylor(b, a.orden))
       ($f)(a::Number, b::Taylor) = ($f)(Taylor(a, b.orden), b)
       ($f)(a::Taylor) = Taylor(($f)(a.coeficientes), a.orden)
    end
end

#Multiplicacion
function *(a::Taylor, b::Taylor)
    a1, b1, orden = fixshape(a, b)
    T = eltype(a1)
    coeficientes = zeros(T,orden+1)
    coeficientes[1] = a1.coeficientes[1] * b1.coeficientes[1]
    for k = 1:orden
        coeficientes[k+1] = mulHomogcoef(k, a1.coeficientes, b1.coeficientes)
    end
    Taylor(coeficientes, orden)
end

#Coeficiente homogeneo para multiplicacion
function mulHomogcoef{T<:Number}(kcoef::Integer, ac::Array{T,1}, bc::Array{T,1})
    coefhomog = zero(T)
    for i = 0:kcoef
        coefhomog += ac[i+1] * bc[kcoef-i+1]
    end
    coefhomog
end
*(a::Taylor, b::Number) = Taylor(b*a.coeficientes, a.orden)
*(a::Number, b::Taylor) = Taylor(a*b.coeficientes, b.orden)

#Division
function /(a::Taylor, b::Taylor)
    a1, b1, orden = fixshape(a, b)
    ordLHopital, cLHopital = divlhopital(a1, b1) # L'Hôpital orden y coeficiente
    T = typeof(cLHopital)
    v1 = convert(Array{T,1}, a1.coeficientes)
    v2 = convert(Array{T,1}, b1.coeficientes)
    coeficientes = zeros(T,orden+1)
    coeficientes[1] = cLHopital
    for k = ordLHopital+1:orden
        coeficientes[k-ordLHopital+1] = divHomogcoef(k, v1, v2, coeficientes, ordLHopital)
    end
    Taylor(coeficientes, orden)
end

# Coeficiente Homogeneo para Division
function divHomogcoef{T<:Number}(kcoef::Integer, ac::Array{T,1}, bc::Array{T,1}, 
        coeficientes::Array{T,1}, ordLHopital::Integer)
    coefhomog = mulHomogcoef(kcoef, coeficientes, bc)
    coefhomog = (ac[kcoef+1]-coefhomog) / bc[ordLHopital+1]
    coefhomog
end
/(a::Taylor,b::Number) = Taylor(a.coeficientes/b, a.orden)
/(a::Number,b::Taylor) = Taylor([a], b.orden) / b



/ (generic function with 51 methods)

In [8]:
#Logaritmo
function log(a::Taylor)
    orden = a.orden
    l0nz = firstnonzero(a)
    if firstnonzero(a)>0
        error("No es posible expander log alrededor de 0.\n")
    end
    aux = log( a.coeficientes[1] )
    T = typeof(aux)
    ac = convert(Array{T,1}, a.coeficientes)
    coeficientes = zeros(T, orden+1)
    coeficientes[1] = aux
    for k = 1:orden
        coeficientes[k+1] = logHomogcoef(k, ac, coeficientes)
    end
    Taylor( coeficientes, orden )
end
#Coeficientes homogeneos para logaritmo
function logHomogcoef{T<:Number}(kcoef::Integer, ac::Array{T,1}, coeficientes::Array{T,1})
  coefhomog = zero(T)
  for i = 1:kcoef-1
    coefhomog += (kcoef-i) * ac[i+1] * coeficientes[kcoef-i+1]
  end
  coefhomog = (ac[kcoef+1] -coefhomog/kcoef) / ac[1]
  coefhomog
end



logHomogcoef (generic function with 1 method)

In [9]:
#Exponente
function ^(a::Taylor, x::Integer)
    uno = one(a)
    if x < 0
        return uno / a^(-x)
    elseif x == 0
        return uno
    elseif x%2 == 0 # par
        if x == 2
            return square(a)
        else
            pow = div(x, 2)
            return square( a^pow )
        end
    else  # impar
        if x == 1
            return a
        else
            expon = div(x-1, 2)
            return a*square( a^expon )
        end
    end
end



^ (generic function with 46 methods)

In [10]:
#Sin y Cos
sin(a::Taylor) = sincos(a, "sin")
cos(a::Taylor) = sincos(a, "cos")
function sincos(a::Taylor, fun::String)
    orden = a.orden
    aux = sin( a.coeficientes[1] )
    T = typeof(aux)
    v = convert(Array{T,1}, a.coeficientes)
    sincoeficientes = zeros(T,orden+1)
    coscoeficientes = zeros(T,orden+1)
    sincoeficientes[1] = aux
    coscoeficientes[1] = cos( a.coeficientes[1] )
    for k = 1:orden
        sincoeficientes[k+1], coscoeficientes[k+1] = sincosHomogcoef(k, v, sincoeficientes, coscoeficientes)
    end
    if fun == "sin"
        return Taylor( sincoeficientes, orden )
    else
        return Taylor( coscoeficientes, orden )
    end
end
#Coeficientes homogeneos para cos y sin
function sincosHomogcoef{T<:Number}(kcoef::Integer, ac::Array{T,1}, 
        sincoeficientes::Array{T,1}, coscoeficientes::Array{T,1})
    sincoefhom = zero(T)
    coscoefhom = zero(T)
    for i = 1:kcoef
        number = i * ac[i+1]
        sincoefhom += number * coscoeficientes[kcoef-i+1]
        coscoefhom -= number * sincoeficientes[kcoef-i+1]
    end
    sincoefhom = sincoefhom/kcoef
    coscoefhom = coscoefhom/kcoef
    return sincoefhom, coscoefhom
end



  likely near In[10]:4


sincosHomogcoef (generic function with 1 method)

In [11]:
#Primer Ejercicio

#Función que calcula los coeficientes x[k]
function coeficientes{T<:Number}(f::Function, p::Int, x_0=3)
    coeft = Taylor(zeros(a.coeficientes), orden+1)
    for k in 0:p
        x[k+1] = f[k]/k+1
    end
    coeft.coeficientes[0] = x_0
    coeft.coeficientes[k] = x[k+1]
end



The method will not be callable.


coeficientes (generic function with 2 methods)

In [18]:
#Función para obtener el paso de integración h

function paso_integ(p::Int)
    x_p = coeft.coeficientes[p]
    h = ((eps/abs(x_p))^(1/p))/2
end
    

paso_integ (generic function with 1 method)

In [19]:
f(x)=x^2

f (generic function with 1 method)

In [20]:
x_0 = 3

3

In [24]:
coeficientes(x^2, 1)

LoadError: LoadError: UndefVarError: x not defined
while loading In[24], in expression starting on line 1

**Ejercicio 2:** Repitan los cálculos usando el método de Runge-Kutta de 4o orden con paso de integración fijo (que es lo más sofisticado que conocen hasta ahora) y comparen los resultados del error relativo con el método de Taylor. En particular, finjan ignorancia de la misma manera que en el ejercicio anterior.

In [ ]:
#Segundo Ejercicio

**Ejercicio 3:** Integra la ecuación de movimiento para el oscilador armónico, $\ddot{x} = -2x$, con $x(0)=2$, $\dot{x}(0)=0$, durante 10^4 periodos de oscilación (o sea, hasta $t_f = 10^4 \cdot 2\pi/\sqrt{2}$, usando el método de Taylor y el método de Runge-Kutta de 4o orden con paso de integración constante. Compara cómo cambia la energía (respecto al valor al tiempo cero) en función del tiempo para ambos métodos.

In [ ]:
#Tercer Ejercicio